In [50]:
from pymed import PubMed
pubmed = PubMed(tool="MyTool", email="my@email.address")
test_str = '"Ephrin-Eph signaling in axon guidance and beyond"[Title:~0]'


In [57]:
from pymed import PubMed

# Initialize PubMed client
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Exact-title proximity query (adjacent words in Title)
title = (
    "Proteome profile of peripheral myelin in healthy mice and in a neuropathy model"
)
query = f'"{title}"[Title:~0]'

# Fetch up to 5 direct hits
articles = list(pubmed.query(query, max_results=5))

# Display results
if not articles:
    print("No direct matches found for the exact title.")
else:
    for art in articles:
        print(f"PMID:    {art.pubmed_id}")
        print(f"Title:   {art.title}")
        print(f"Journal: {art.journal}")

        # Authors
        if art.authors:
            authors = [f"{a.get('firstname', '').strip()} {a.get('lastname', '').strip()}".strip()
                       for a in art.authors]
            print("Authors: ", ", ".join(authors))
        else:
            print("Authors: None listed")

        # Publication year
        year = getattr(art.publication_date, 'year', None)
        print(f"Year:    {year if year else 'Not available'}")
        print("-")


PMID:    32130108
29367382
7581450
23794183
30392800
7685397
25110935
18410371
25195104
17576798
26439954
15474366
16856148
16794783
15557122
8917582
12871592
10397775
25625223
803856
15014110
19026747
17397833
3110266
29546717
14638863
18803319
24038174
14644171
20448149
15356632
16369933
15282162
22441408
24038504
25327648
24336358
27010757
31493234
25762662
27774063
19459211
30820892
30672734
29136549
7698985
24903685
11777940
25686621
12118253
6168745
30072689
23152929
7515849
22622581
19629567
9247276
19294628
28636612
1384988
8155317
10839370
21401967
20014276
28134254
4697881
25274820
11157804
17208176
17549680
13104987
18925949
14715942
28230289
22291039
19452287
31453649
8987820
16534116
25821719
27364987
27629714
25449862
25545627
28248254
18228513
15695336
25720167
8200428
17437540
22801498
25429154
12372015
19294629
2257482
19345204
26272917
21079185
27284195
7540676
11869039
6255099
15044753
666897
11968057
15883201
19745155
21613327
25921593
24598164
23988427
25447940
168

In [66]:
import re
from pymed import PubMed

# Initialize PubMed client (no logging or sleeping)
pubmed = PubMed(tool="MyTool", email="my@email.address")

def replace_entry_noio(match):
    # 1. Extract matched groups
    year_str, authors_str, original_title, journal_str, orig_summary = (
        match.group(1).strip(),
        match.group(2).strip(),
        match.group(3).strip(),
        match.group(4).strip(),
        match.group(5).strip()
    )

    # 2. Exact-title proximity query for direct hit
    query = f'"{original_title}"[Title:~0]'
    try:
        results = list(pubmed.query(query, max_results=1))
    except Exception:
        results = []

    if results:
        art = results[0]
        # Override year if available
        year = getattr(art.publication_date, 'year', year_str)
        # Override journal if available
        journal = art.journal or journal_str
        # Override title itself
        title = art.title
        # Build author label (FirstAuthor et al.)
        if art.authors:
            first = art.authors[0]
            last = first.get('lastname', '').strip()
            author_label = f"{last} et al." if len(art.authors) > 1 else last
        else:
            author_label = authors_str
    else:
        # No match: keep all original values
        year = year_str
        author_label = authors_str
        journal = journal_str
        title = original_title

    # 3. Build citation using the (possibly) overridden title
    citation_str = f'({year}, {author_label}, "{title}", {journal})'
    # 4. Return the new citation plus the summary
    return f"{citation_str} {orig_summary}"


# === Test case ===
summary = (
    '(2010, Fledrich et al., '
    '"Proteome and profile of peripheral myelin in healthy mice and in a neuropathy model", '
    'test) > “PMP22 is essential for the formation and maintenance of peripheral myelin sheaths in Schwann cells.”'
)

# Regex matches (year, authors, title, journal, rest)
pattern = re.compile(
    r'\(\s*(\d{4})\s*,\s*'          # (year,
    r'([^,]+?)\s*,\s*'              #  authors,
    r'"([^"\)]+)"\s*,\s*'           #  "title",
    r'([^\)]+)\)\s*>\s*(.+)'        #  journal) > rest
)

m = pattern.match(summary)
if m:
    print("Before:", summary)
    print("After: ", replace_entry_noio(m))
else:
    print("✘ Regex didn’t match summary format.")


Before: (2010, Fledrich et al., "Proteome and profile of peripheral myelin in healthy mice and in a neuropathy model", test) > “PMP22 is essential for the formation and maintenance of peripheral myelin sheaths in Schwann cells.”
After:  (2020, Siems et al., "Proteome profile of peripheral myelin in healthy mice and in a neuropathy model.", eLife) “PMP22 is essential for the formation and maintenance of peripheral myelin sheaths in Schwann cells.”
